In [1]:
import utils
import requests

text_by_file = utils.load_files('./ustawy/*')

1. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the law corpus.

In [60]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

for file,text in text_by_file.items():
    text_by_file[file] = ' '.join(text.split())
    

nlp = Polish()
tokenizer = nlp.tokenizer
files = tokenizer.pipe(text_by_file.values())

2. Compute **bigram** counts of downcased tokens.  Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   1. "the quick": 1
   1. "quick brown": 1
   1. "brown fox": 1
   1. ...
   1. "dog .": 1

In [30]:
from collections import Counter

bigrams = []

for doc in files:
    words = [token.text.lower() for token in doc]
    bigrams += list(zip(words[:-1],words[1:]))

freq = Counter(bigrams)
freq

Counter({('dz', '.'): 8885,
         ('.', 'u'): 8174,
         ('u', '.'): 8134,
         ('.', 'z'): 5404,
         ('z', '2000'): 1506,
         ('2000', 'r'): 2271,
         ('r', '.'): 33015,
         ('.', 'nr'): 21433,
         ('nr', '31'): 175,
         ('31', ','): 285,
         (',', 'poz'): 43188,
         ('poz', '.'): 45198,
         ('.', '381'): 17,
         ('381', 'ustawa'): 1,
         ('ustawa', 'z'): 1190,
         ('z', 'dnia'): 9527,
         ('dnia', '2'): 149,
         ('2', 'marca'): 22,
         ('marca', '2000'): 43,
         ('.', 'o'): 7693,
         ('o', 'zmianie'): 1299,
         ('zmianie', 'ustawy'): 850,
         ('ustawy', 'o'): 1444,
         ('o', 'przeciwdziałaniu'): 109,
         ('przeciwdziałaniu', 'praktykom'): 29,
         ('praktykom', 'monopolistycznym'): 38,
         ('monopolistycznym', 'i'): 12,
         ('i', 'ochronie'): 22,
         ('ochronie', 'interesów'): 12,
         ('interesów', 'konsumentów'): 53,
         ('konsumentów', 'ar

3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries **after**
   computing the bigram counts.

In [31]:
freq = dict(filter(lambda elem: elem[0][0].isalpha() and elem[0][1].isalpha(), freq.items()))
freq

{('ustawa', 'z'): 1190,
 ('z', 'dnia'): 9527,
 ('o', 'zmianie'): 1299,
 ('zmianie', 'ustawy'): 850,
 ('ustawy', 'o'): 1444,
 ('o', 'przeciwdziałaniu'): 109,
 ('przeciwdziałaniu', 'praktykom'): 29,
 ('praktykom', 'monopolistycznym'): 38,
 ('monopolistycznym', 'i'): 12,
 ('i', 'ochronie'): 22,
 ('ochronie', 'interesów'): 12,
 ('interesów', 'konsumentów'): 53,
 ('konsumentów', 'art'): 3,
 ('w', 'ustawie'): 5180,
 ('ustawie', 'z'): 3661,
 ('w', 'art'): 32042,
 ('wprowadza', 'się'): 1849,
 ('się', 'następujące'): 1855,
 ('następujące', 'zmiany'): 1808,
 ('otrzymuje', 'brzmienie'): 9553,
 ('prezes', 'jest'): 4,
 ('jest', 'centralnym'): 26,
 ('centralnym', 'organem'): 51,
 ('organem', 'administracji'): 85,
 ('administracji', 'rządowej'): 988,
 ('po', 'ust'): 1541,
 ('dodaje', 'się'): 8195,
 ('się', 'ust'): 3484,
 ('w', 'brzmieniu'): 7280,
 ('prezes', 'rady'): 602,
 ('rady', 'ministrów'): 1168,
 ('ministrów', 'sprawuje'): 7,
 ('sprawuje', 'nadzór'): 132,
 ('nadzór', 'nad'): 517,
 ('nad', 'dzia

4. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure 
   for all pairs of words. 

In [34]:
words = []

for doc in files:
    words += [token.text.lower() for token in doc if token.text.isalpha()]

word_freq = Counter(words)
word_freq

Counter({'dz': 8885,
         'u': 9153,
         'z': 82443,
         'r': 33177,
         'nr': 44950,
         'poz': 45224,
         'ustawa': 3235,
         'dnia': 17954,
         'marca': 783,
         'o': 64776,
         'zmianie': 1518,
         'ustawy': 13099,
         'przeciwdziałaniu': 216,
         'praktykom': 43,
         'monopolistycznym': 39,
         'i': 90009,
         'ochronie': 1054,
         'interesów': 236,
         'konsumentów': 328,
         'art': 83804,
         'w': 201223,
         'ustawie': 5478,
         'lutego': 608,
         'wprowadza': 1999,
         'się': 45886,
         'następujące': 2276,
         'zmiany': 4016,
         'a': 17133,
         'ust': 53636,
         'otrzymuje': 9835,
         'brzmienie': 10576,
         'prezes': 2497,
         'jest': 13197,
         'centralnym': 129,
         'organem': 683,
         'administracji': 2941,
         'rządowej': 1097,
         'b': 6849,
         'po': 13547,
         'dodaje': 8423,


In [35]:
word_total = sum(word_freq.values())
bigram_total = sum(freq.values())

In [55]:
from math import log2

def get_pmi(wf,bf):
    wt = sum(wf.values())
    bt = sum(bf.values())
    def pmi(a: str, b: str) -> float:
        pa = wf[a]/wt
        pb = wf[b]/wt
        pab = bf[(a,b)]/bt
        return log2(pab/(pa*pb))
    return pmi

In [37]:
pmi = get_pmi(word_freq, freq)
bigram_pmi = {bigram: pmi(*bigram) for bigram in freq.keys()}
bigram_pmi

{('ustawa', 'z'): 4.375256412169623,
 ('z', 'dnia'): 4.9038491447023995,
 ('o', 'zmianie'): 5.9412278119895365,
 ('zmianie', 'ustawy'): 7.635363832038612,
 ('ustawy', 'o'): 2.9846841411281777,
 ('o', 'przeciwdziałaniu'): 5.179294994570579,
 ('przeciwdziałaniu', 'praktykom'): 13.825998672407135,
 ('praktykom', 'monopolistycznym'): 16.68543047402437,
 ('monopolistycznym', 'i'): 3.9909485727211806,
 ('i', 'ochronie'): 0.10916075786235552,
 ('ochronie', 'interesów'): 7.809830233867229,
 ('interesów', 'konsumentów'): 11.636895334728404,
 ('konsumentów', 'art'): -0.9781510553075033,
 ('w', 'ustawie'): 4.450037075451804,
 ('ustawie', 'z'): 5.236649058453354,
 ('w', 'art'): 3.1436793383832353,
 ('wprowadza', 'się'): 6.550870258441064,
 ('się', 'następujące'): 6.368322134983079,
 ('następujące', 'zmiany'): 9.845520500302962,
 ('otrzymuje', 'brzmienie'): 8.738692284743909,
 ('prezes', 'jest'): -0.8247325584143633,
 ('jest', 'centralnym'): 6.150460010231687,
 ('centralnym', 'organem'): 11.3946262

5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [38]:
top_10 = list(sorted(bigram_pmi.items(), key=lambda item: item[1], reverse=True))[:10]
top_10

[(('młody', 'naukowiec'), 22.149169934145128),
 (('młodym', 'naukowcom'), 22.149169934145128),
 (('osiągniętymi', 'efektami'), 22.149169934145128),
 (('del', 'credere'), 22.149169934145128),
 (('doktorów', 'habilitowanych'), 22.149169934145128),
 (('fenolami', 'lotnymi'), 22.149169934145128),
 (('ostrzeżony', 'strzałami'), 22.149169934145128),
 (('strzałami', 'ostrzegawczymi'), 22.149169934145128),
 (('płynami', 'infuzyjnymi'), 22.149169934145128),
 (('dzierżawionego', 'obowdu'), 22.149169934145128)]

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5
   occurrences).

In [39]:
bigram_5_pmi = {bigram: pmi(*bigram) for (bigram,count) in freq.items() if count > 4}
top_10_5 = list(sorted(bigram_5_pmi.items(), key=lambda item: item[1], reverse=True))[:10]
top_10_5

[(('obiegów', 'chłodzących'), 19.827241839257766),
 (('otworami', 'wiertniczymi'), 19.827241839257766),
 (('ręcznego', 'miotacza'), 19.827241839257766),
 (('świeckie', 'przygotowujące'), 19.827241839257766),
 (('stajnią', 'wyścigową'), 19.827241839257766),
 (('młynki', 'młotkowe'), 19.827241839257766),
 (('młyny', 'kulowe'), 19.827241839257766),
 (('najnowszych', 'zdobyczy'), 19.827241839257766),
 (('klęskami', 'żywiołowymi'), 19.827241839257766),
 (('zaszkodzić', 'wynikom'), 19.827241839257766)]

7. Use [KRNNT](https://hub.docker.com/r/djstrong/krnnt2) or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.

In [46]:
import xml.etree.ElementTree as ET
import glob

def parse_xml(path: str) -> list[str]:
    tree = ET.parse(path)
    root = tree.getroot()

    words = []
    
    for tok in root.iter('tok'):
        lex = tok.find('lex')
        base = lex.find('base').text
        ctag = lex.find('ctag').text.split(':')[0]
        words.append(f'{base}:{ctag}')
    
    return words

def parse_files(path: str) -> dict[str, str]:
    files = []

    for file in glob.glob(path):
        files.append(parse_xml(file))

    return files

files_xml = parse_files('./ustawy_lem/*')


8. Using the tagged corpus compute bigram statistic for the tokens containing:
   a. lemmatized, downcased word
   b. morphosyntactic category of the word (subst, fin, adj, etc.)
9. For example: "Ala ma kota", which is tagged as:
   ```
   Ala	none
           Ala	subst:sg:nom:f	disamb
   ma	space
           mieć	fin:sg:ter:imperf	disamb
   kota	space
           kot	subst:sg:acc:m2	disamb
   .	none
           .	interp	disamb
   ```
   the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

In [50]:
bigrams_xml = []
words_xml = []

for doc in files_xml:
    words = [token.lower() for token in doc]
    words_xml += list(filter(lambda w: w.split(':')[0].isalpha(), words))
    bigrams_xml += list(zip(words[:-1],words[1:]))

freq_xml = Counter(bigrams_xml)
freq_xml = dict(filter(lambda elem: elem[0][0].split(':')[0].isalpha() and elem[0][1].split(':')[0].isalpha(), freq_xml.items()))
word_xml_freq = Counter(words_xml)
freq_xml

{('ustawa:subst', 'z:prep'): 8625,
 ('z:prep', 'dzień:subst'): 11360,
 ('o:prep', 'emerytura:subst'): 189,
 ('emerytura:subst', 'i:conj'): 386,
 ('i:conj', 'renta:subst'): 388,
 ('renta:subst', 'z:prep'): 284,
 ('z:prep', 'fundusz:subst'): 409,
 ('fundusz:subst', 'ubezpieczenie:subst'): 182,
 ('ubezpieczenie:subst', 'społeczny:adj'): 1668,
 ('społeczny:adj', 'dział:subst'): 1,
 ('dział:subst', 'i:conj'): 109,
 ('i:conj', 'przepis:subst'): 134,
 ('przepis:subst', 'ogólny:adj'): 255,
 ('ogólny:adj', 'rozdział:subst'): 8,
 ('zakres:subst', 'podmiotowy:adj'): 12,
 ('podmiotowy:adj', 'i:conj'): 5,
 ('i:conj', 'przedmiotowy:adj'): 4,
 ('przedmiotowy:adj', 'ustawa:subst'): 1,
 ('ustawa:subst', 'art:ign'): 117,
 ('ustawa:subst', 'określać:fin'): 217,
 ('warunek:subst', 'nabywać:ger'): 12,
 ('nabywać:ger', 'prawa:subst'): 8,
 ('prawa:subst', 'do:prep'): 836,
 ('do:prep', 'świadczenie:subst'): 324,
 ('świadczenie:subst', 'pieniężny:adj'): 296,
 ('pieniężny:adj', 'z:prep'): 134,
 ('z:prep', 'ubez

10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [56]:
pmi_xml = get_pmi(word_xml_freq, freq_xml)
bigram_5_pmi_xml = {bigram: pmi_xml(*bigram) for (bigram,count) in freq_xml.items() if count > 4}
top_10_5_xml = list(sorted(bigram_5_pmi_xml.items(), key=lambda item: item[1], reverse=True))[:10]
top_10_5_xml

[(('grzegorz:subst', 'schetyna:ign'), 19.82364459286172),
 (('młynek:subst', 'młotkowy:adj'), 19.82364459286172),
 (('teryto:ign', 'rialnego:ign'), 19.82364459286172),
 (('pasta:subst', 'emulsyjny:adj'), 19.560610187027926),
 (('adam:subst', 'mickiewicz:subst'), 19.560610187027926),
 (('chrom:subst', 'sześciowartościowy:adj'), 19.560610187027926),
 (('odpowiedzieć:fin', 'dzialności:ign'), 19.560610187027926),
 (('łańcuchowa:subst', 'rozszczepienie:subst'), 19.560610187027926),
 (('młyn:subst', 'kulowy:adj'), 19.338217765691475),
 (('piotrek:subst', 'trybunalski:adj'), 19.338217765691475)]

11. Compute **trigram** counts for both corpora and perform the same filtering.

In [61]:
trigrams = []

for doc in files:
    words = [token.text.lower() for token in doc]
    trigrams += list(zip(words[:-2],words[1:-1],words[2:]))

freq_tri = Counter(trigrams)
freq_tri

Counter({('dz', '.', 'u'): 8151,
         ('.', 'u', '.'): 8133,
         ('u', '.', 'z'): 4510,
         ('.', 'z', '2000'): 351,
         ('z', '2000', 'r'): 1506,
         ('2000', 'r', '.'): 2270,
         ('r', '.', 'nr'): 17860,
         ('.', 'nr', '31'): 112,
         ('nr', '31', ','): 172,
         ('31', ',', 'poz'): 172,
         (',', 'poz', '.'): 43166,
         ('poz', '.', '381'): 13,
         ('.', '381', 'ustawa'): 1,
         ('381', 'ustawa', 'z'): 1,
         ('ustawa', 'z', 'dnia'): 1190,
         ('z', 'dnia', '2'): 141,
         ('dnia', '2', 'marca'): 19,
         ('2', 'marca', '2000'): 6,
         ('marca', '2000', 'r'): 43,
         ('r', '.', 'o'): 7079,
         ('.', 'o', 'zmianie'): 859,
         ('o', 'zmianie', 'ustawy'): 849,
         ('zmianie', 'ustawy', 'o'): 642,
         ('ustawy', 'o', 'przeciwdziałaniu'): 11,
         ('o', 'przeciwdziałaniu', 'praktykom'): 28,
         ('przeciwdziałaniu', 'praktykom', 'monopolistycznym'): 29,
         ('prakt

In [63]:
freq_tri = dict(filter(lambda elem: elem[0][0].isalpha() and elem[0][1].isalpha() and elem[0][2].isalpha(), freq_tri.items()))
freq_tri

{('ustawa', 'z', 'dnia'): 1190,
 ('o', 'zmianie', 'ustawy'): 849,
 ('zmianie', 'ustawy', 'o'): 642,
 ('ustawy', 'o', 'przeciwdziałaniu'): 11,
 ('o', 'przeciwdziałaniu', 'praktykom'): 28,
 ('przeciwdziałaniu', 'praktykom', 'monopolistycznym'): 29,
 ('praktykom', 'monopolistycznym', 'i'): 12,
 ('monopolistycznym', 'i', 'ochronie'): 12,
 ('i', 'ochronie', 'interesów'): 12,
 ('ochronie', 'interesów', 'konsumentów'): 12,
 ('interesów', 'konsumentów', 'art'): 1,
 ('w', 'ustawie', 'z'): 3645,
 ('ustawie', 'z', 'dnia'): 3649,
 ('wprowadza', 'się', 'następujące'): 1804,
 ('się', 'następujące', 'zmiany'): 1806,
 ('prezes', 'jest', 'centralnym'): 2,
 ('jest', 'centralnym', 'organem'): 26,
 ('centralnym', 'organem', 'administracji'): 51,
 ('organem', 'administracji', 'rządowej'): 55,
 ('dodaje', 'się', 'ust'): 2766,
 ('prezes', 'rady', 'ministrów'): 601,
 ('rady', 'ministrów', 'sprawuje'): 6,
 ('ministrów', 'sprawuje', 'nadzór'): 7,
 ('sprawuje', 'nadzór', 'nad'): 116,
 ('nadzór', 'nad', 'działaln

In [62]:
trigrams_xml = []

for doc in files_xml:
    words = [token.lower() for token in doc]
    trigrams_xml += list(zip(words[:-2],words[1:-1],words[2:]))

freq_tri_xml = Counter(trigrams_xml)
freq_tri_xml

Counter({('dzieje_(apostolskie):brev', '.:interp', 'u:prep'): 5776,
         ('.:interp', 'u:prep', '.:interp'): 8135,
         ('u:prep', '.:interp', 'z:prep'): 4512,
         ('.:interp', 'z:prep', '1998:num'): 270,
         ('z:prep', '1998:num', 'r:ign'): 1492,
         ('1998:num', 'r:ign', '.:interp'): 2323,
         ('r:ign', '.:interp', 'nr:subst'): 17860,
         ('.:interp', 'nr:subst', '162:num'): 188,
         ('nr:subst', '162:num', ',:interp'): 636,
         ('162:num', ',:interp', 'poz:ign'): 635,
         (',:interp', 'poz:ign', '.:interp'): 43172,
         ('poz:ign', '.:interp', '1118:num'): 263,
         ('.:interp', '1118:num', 'ustawa:subst'): 3,
         ('1118:num', 'ustawa:subst', 'z:prep'): 3,
         ('ustawa:subst', 'z:prep', 'dzień:subst'): 8589,
         ('z:prep', 'dzień:subst', '17:num'): 380,
         ('dzień:subst', '17:num', 'grudzień:subst'): 111,
         ('17:num', 'grudzień:subst', '1998:num'): 71,
         ('grudzień:subst', '1998:num', 'r:ign')

In [64]:

freq_tri_xml = dict(filter(lambda elem: elem[0][0].split(':')[0].isalpha() and elem[0][1].split(
    ':')[0].isalpha() and elem[0][2].split(':')[0].isalpha(), freq_tri_xml.items()))
freq_tri_xml


{('ustawa:subst', 'z:prep', 'dzień:subst'): 8589,
 ('o:prep', 'emerytura:subst', 'i:conj'): 163,
 ('emerytura:subst', 'i:conj', 'renta:subst'): 378,
 ('i:conj', 'renta:subst', 'z:prep'): 168,
 ('renta:subst', 'z:prep', 'fundusz:subst'): 134,
 ('z:prep', 'fundusz:subst', 'ubezpieczenie:subst'): 146,
 ('fundusz:subst', 'ubezpieczenie:subst', 'społeczny:adj'): 181,
 ('ubezpieczenie:subst', 'społeczny:adj', 'dział:subst'): 1,
 ('społeczny:adj', 'dział:subst', 'i:conj'): 1,
 ('dział:subst', 'i:conj', 'przepis:subst'): 31,
 ('i:conj', 'przepis:subst', 'ogólny:adj'): 36,
 ('przepis:subst', 'ogólny:adj', 'rozdział:subst'): 2,
 ('zakres:subst', 'podmiotowy:adj', 'i:conj'): 4,
 ('podmiotowy:adj', 'i:conj', 'przedmiotowy:adj'): 4,
 ('i:conj', 'przedmiotowy:adj', 'ustawa:subst'): 1,
 ('przedmiotowy:adj', 'ustawa:subst', 'art:ign'): 1,
 ('warunek:subst', 'nabywać:ger', 'prawa:subst'): 8,
 ('nabywać:ger', 'prawa:subst', 'do:prep'): 8,
 ('prawa:subst', 'do:prep', 'świadczenie:subst'): 91,
 ('do:prep'

12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the
   results for bigrams.

In [66]:
def get_pmi_tri(wf,bf,tf):
    wt = sum(wf.values())
    bt = sum(bf.values())
    tt = sum(tf.values())
    def pmi(a: str, b: str,c: str) -> float:
        pa = wf[a]/wt
        pbc = bf[(b,c)]/bt
        pabc = tf[(a,b,c)]/tt
        return log2(pabc/(pa*pbc))
    return pmi

In [67]:
pmi_tri = get_pmi_tri(word_freq, freq, freq_tri)
trigram_5_pmi = {trigram: pmi_tri(*trigram) for (trigram,count) in freq_tri.items() if count > 4}
tri_top_10_5 = list(sorted(trigram_5_pmi.items(), key=lambda item: item[1], reverse=True))[:10]
tri_top_10_5

[(('niemieckiego', 'towarzystwa', 'wspierania'), 19.744611281299488),
 (('chwytów', 'obezwładniających', 'oraz'), 19.744611281299488),
 (('konduktów', 'pogrzebowych', 'odbywających'), 19.744611281299488),
 (('świeckie', 'przygotowujące', 'się'), 19.744611281299488),
 (('naliczeń', 'etatowych', 'w'), 19.744611281299488),
 (('emeryci', 'i', 'renciści'), 19.744611281299488),
 (('brat', 'lub', 'siostra'), 19.744611281299488),
 (('byłe', 'mieszkanie', 'zakładowe'), 19.744611281299488),
 (('fiskalne', 'i', 'rejestrujące'), 19.744611281299488),
 (('najnowszych', 'zdobyczy', 'techniki'), 19.744611281299488)]

In [69]:
pmi_tri_xml = get_pmi_tri(word_xml_freq, freq_xml, freq_tri_xml)
trigram_5_pmi_xml = {trigram: pmi_tri_xml(*trigram) for (trigram,count) in freq_tri_xml.items() if count > 4}
tri_top_10_5_xml = list(sorted(trigram_5_pmi_xml.items(), key=lambda item: item[1], reverse=True))[:10]
tri_top_10_5_xml

[(('chwyt:subst', 'obezwładniający:adj', 'oraz:conj'), 19.743148030139597),
 (('pięciodobowy:adj', 'biochemiczny:adj', 'zapotrzebowanie:subst'),
  19.743148030139597),
 (('kondukt:subst', 'pogrzebowy:adj', 'odbywać:pact'), 19.743148030139597),
 (('obrabiarka:subst', 'do:prep', 'metal:subst'), 19.743148030139597),
 (('piaskownia:subst', 'przemysł:subst', 'węglowy:adj'), 19.743148030139597),
 (('nożyca:subst', 'hutniczy:adj', 'do:prep'), 19.743148030139597),
 (('chemoodporny:adj', 'dla:prep', 'kwas:subst'), 19.743148030139597),
 (('przejezdna:subst', 'oraz:conj', 'nieprzejezdna:subst'),
  19.743148030139597),
 (('czechosłowacki:adj', 'republika:subst', 'socjalistyczny:adj'),
  19.743148030139597),
 (('luk:subst', 'w:prep', 'współczesny:adj'), 19.743148030139597)]

13. Create a table comparing the results for copora without and with tagging and lemmatization (separate table for bigrams and trigrams).

In [72]:
import pandas as pd
top_10_5_keys, top_10_5_values = zip(*top_10_5)
top_10_5_xml_keys, top_10_5_xml_values = zip(*top_10_5_xml)

pd.DataFrame({ 'spacy keys' :top_10_5_keys, 'spacy values': top_10_5_values, 'clarin keys': top_10_5_xml_keys, 'clarin values': top_10_5_xml_values})

,spacy keys,spacy values,clarin keys,clarin values
0,"(obiegów, chłodzących)",19.827242,"(grzegorz:subst, schetyna:ign)",19.823645
1,"(otworami, wiertniczymi)",19.827242,"(młynek:subst, młotkowy:adj)",19.823645
2,"(ręcznego, miotacza)",19.827242,"(teryto:ign, rialnego:ign)",19.823645
3,"(świeckie, przygotowujące)",19.827242,"(pasta:subst, emulsyjny:adj)",19.560610
4,"(stajnią, wyścigową)",19.827242,"(adam:subst, mickiewicz:subst)",19.560610
5,"(młynki, młotkowe)",19.827242,"(chrom:subst, sześciowartościowy:adj)",19.560610
6,"(młyny, kulowe)",19.827242,"(odpowiedzieć:fin, dzialności:ign)",19.560610
7,"(najnowszych, zdobyczy)",19.827242,"(łańcuchowa:subst, rozszczepienie:subst)",19.560610
8,"(klęskami, żywiołowymi)",19.827242,"(młyn:subst, kulowy:adj)",19.338218
9,"(zaszkodzić, wynikom)",19.827242,"(piotrek:subst, trybunalski:adj)",19.338218


In [73]:
top_10_5_tri_keys, top_10_5_tri_values = zip(*tri_top_10_5)
top_10_5_tri_xml_keys, top_10_5_tri_xml_values = zip(*tri_top_10_5_xml)

pd.DataFrame({ 'spacy keys' :top_10_5_tri_keys, 'spacy values': top_10_5_tri_values, 'clarin keys': top_10_5_tri_xml_keys, 'clarin values': top_10_5_tri_xml_values})

,spacy keys,spacy values,clarin keys,clarin values
0,"(niemieckiego, towarzystwa, wspierania)",19.744611,"(chwyt:subst, obezwładniający:adj, oraz:conj)",19.743148
1,"(chwytów, obezwładniających, oraz)",19.744611,"(pięciodobowy:adj, biochemiczny:adj, zapotrzeb...",19.743148
2,"(konduktów, pogrzebowych, odbywających)",19.744611,"(kondukt:subst, pogrzebowy:adj, odbywać:pact)",19.743148
3,"(świeckie, przygotowujące, się)",19.744611,"(obrabiarka:subst, do:prep, metal:subst)",19.743148
4,"(naliczeń, etatowych, w)",19.744611,"(piaskownia:subst, przemysł:subst, węglowy:adj)",19.743148
5,"(emeryci, i, renciści)",19.744611,"(nożyca:subst, hutniczy:adj, do:prep)",19.743148
6,"(brat, lub, siostra)",19.744611,"(chemoodporny:adj, dla:prep, kwas:subst)",19.743148
7,"(byłe, mieszkanie, zakładowe)",19.744611,"(przejezdna:subst, oraz:conj, nieprzejezdna:su...",19.743148
8,"(fiskalne, i, rejestrujące)",19.744611,"(czechosłowacki:adj, republika:subst, socjalis...",19.743148
9,"(najnowszych, zdobyczy, techniki)",19.744611,"(luk:subst, w:prep, współczesny:adj)",19.743148


14. Answer the following questions:
      1. Why do we have to filter the bigrams, rather than the token sequence?
      1. Which method works better for the bigrams and which for the trigrams?
      1. What types of expressions are discovered by the methods.
      1. Can you devise a different type of filtering that would yield better results?